## Calibrate an implied volatility surface - SSVI

Last Updated: **2024-07-05**  
This notebook can be cloned from [my GitHub -> research -> articles](https://github.com/se-l/research)

There are numerous papers on calibrating implied volatility surface. Here, I take a look at SSVI applied to a day of FDX just before and after their corporate earnings announcement. I want to see the quality of fit and how the model parameters behave throughout the day and across earnings announcement. Regarding fit, I would want:  
- A model that fits actual fill IVs and is always within bid/ask.
- Fitting error plotted with respect to tenor and moneyness.

SSVI stands for Surface Stochastic Volatility Inspired. The approach has been applied from: [No arbitrage global parametrization for the eSSVI volatility, A. Mingon 2022](https://arxiv.org/abs/2204.00312) which itself is extended from extended from [The extended SSVI volatility surface, Hendriks, Martini 2017](https://papers.ssrn.com/sol3/papers.cfm?abstract_id=2971502) and Gathedral (2012).

A slice of the surface's total implied variance is defined as:  
$
eSSVI(K, T) = \dfrac{1}{2} (\theta(T) + \rho(T)\psi(T)k + \sqrt{(\psi(T)k + \theta(T)\rho(T))^2 + \theta(T)^2(1-\rho(T)^2)})
$

where the log-forward-moneyness $
k = log \dfrac{K}{F_0(T)}
$, with $K$ being the option strike and $F_0$(T) the forward.

and the implied volatility can be recovered with  
$
\sigma_{imp}(K, T) = \sqrt{\dfrac{eSSVI(K,T)}{T}}
$

Let's start minimizing for those slice-wise equations without *global* surface optimization, so constraints to exclude calendar arbitrage.
 ρ2) .
e

Feel free to skip the code. It:  
- Imports few bits so you can reproduce this
- Loads & scopes data. IVs have been pre-calculated. This is not about using QuantLib now
- Calibrates with a custom IV surface class that uses scipy.optimize for a least-squares optimizing minimizing RMSE of fill IVs vs the model.

In [1]:
import sys, os
from pathlib import Path
sys.path.append(str(Path(os.getcwd()).parent.parent))

In [2]:
import numpy as np
import pickle
import QuantLib as ql
import pandas as pd

from typing import List
from datetime import timedelta, date, time
from IPython.display import IFrame

from options.client import Client
from options.helper import get_dividend_yield, historical_volatility, get_tenor, unpack_mi_df_index
from options.typess.calibration_item import CalibrationItem, df2calibration_items
from options.typess.enums import Resolution, SecurityType, OptionRight, TickType
from options.typess.equity import Equity
from options.typess.iv_surface_essvi import CalibrationItem, IVSurface
from shared.constants import EarningsPreSessionDates, DiscountRateMarket
from shared.plotting import plot_scatter_3d
from options.typess.equity import Equity

release_date = date(2024, 6, 25)
equity = Equity('FDX')
moneyness_fit = 'moneyness_fwd_ln'
iv_col_nm = 'mid_price_iv'
weight_col_nm = 'volume'
min_mny = -0.3
max_mny = 0.3
print(release_date)

2024-06-25


##### Load & Scope

In [3]:
df = pd.read_excel('df_quotes.xlsx')
df['expiry'] = df['expiry'].apply(lambda x: x.date() if isinstance(x, pd.Timestamp) else x)
df = df.set_index(['ts', 'expiry', 'strike', 'right'])

df_trades = pd.read_excel('df_trades.xlsx')
df_trades['expiry'] = df_trades['expiry'].apply(lambda x: x.date() if isinstance(x, pd.Timestamp) else x)
df_trades = df_trades.set_index(['ts', 'expiry', 'strike', 'right'])

In [4]:
release_date_p1 = [d.date() for d in (pd.date_range(release_date, release_date + timedelta(days=1), freq='D'))][-1]

# Scope by time of day excluding 0:30 -> 10
v_ts = df.index.get_level_values('ts').unique()
v_ts_pre_release = v_ts[(v_ts.date == release_date) & (v_ts.hour >= 10) & (v_ts.hour < 16)]
v_ts_post_release = v_ts[(v_ts.date == release_date_p1) & (v_ts.hour >= 10) & (v_ts.hour < 16)]

vt_ts = df_trades.index.get_level_values('ts').unique()
vt_ts_pre_release = vt_ts[(vt_ts.date == release_date) & (vt_ts.hour >= 10) & (vt_ts.hour < 16)]
vt_ts_post_release = vt_ts[(vt_ts.date == release_date_p1) & (vt_ts.hour >= 10) & (vt_ts.hour < 16)]

df_q_pre = df.loc[v_ts_pre_release]
df_q_post = df.loc[v_ts_post_release]
df_t_pre = df_trades.loc[vt_ts_pre_release]
df_t_post = df_trades.loc[vt_ts_post_release]

##### Calibrate

In [5]:
%%capture
def scope_df(x: pd.DataFrame):
    return x[(x[moneyness_fit] < max_mny) & (x[moneyness_fit] > min_mny)]

calibration_items_fill = df2calibration_items(scope_df(df_t_pre), release_date, y_col_nm='fill_iv', weight_col_nm=weight_col_nm, vega_col_nm='vega_fill_iv')
ivs_t0 = IVSurface(equity).calibrate(calibration_items_fill, verbose=0)

calibration_items_bid = df2calibration_items(scope_df(df_q_pre), release_date, y_col_nm='bid_iv', weight_col_nm='vega_mid_iv')
calibration_items_ask = df2calibration_items(scope_df(df_q_pre), release_date, y_col_nm='ask_iv', weight_col_nm='vega_mid_iv')

2024-07-26 14:20:46,232 - INFO Calibration done for FDX total_rmse=0.6960506299548888


##### Plot
I used actual trades between 10am and 3pm. One may want to use snaps, shorter timeperiods, moving windows (getting to that later)  

The whole surface:  

In [11]:
fn = 'ivs.html'
ivs_t0.plot_surface(fn=fn, open_browser=False)
IFrame(src='figures/' + fn, width='100%', height=800)

A well matching slice. Weight is the respective trade volume. Previously, I used vega.

In [12]:
# ix_best_slice = ivs_t0.ps_rmse().index[ivs_t0.ps_rmse().argmax()]
ix_best_slice = (date(2024, 9, 20), 'call')
fn='ok_smile.html'
ivs_t0.plot_smile(*ix_best_slice, release_date, calibration_items_bid=calibration_items_bid, calibration_items_ask=calibration_items_ask, fn=fn, open_browser=False)
IFrame(src='figures/' + fn, width='100%', height=600)

Now an example for a plot not well matching. Too often, the model is parabolic during a fairly linear section leading to over and under estimation making it infeasible to use for pricing.

In [13]:
# ix_worst_slice = ivs_t0.ps_rmse().index[ivs_t0.ps_rmse().argmax()]
ix_worst_slice = (date(2024, 7, 5), 'put')
fn='bad_smile.html'
ivs_t0.plot_smile(*ix_worst_slice, release_date, calibration_items_bid=calibration_items_bid, calibration_items_ask=calibration_items_ask, fn=fn, open_browser=False)
IFrame(src='figures/' + fn, width='100%', height=600)

How to the fitting parameter behave across tenors? 
- The fitting error is clearly very elevated for the shortest expiry. Note, this data is the last trading before a quarterly earnings release.
- An arbitrage free conditions is for theta to always increase. That looks good. Other conditioned will be investigated later. 

In [14]:
fn = 'ivs_t0_params_rmse.html'
ivs_t0.plot_params_rmse(release_date, fn=fn, open_browser=False)
IFrame(src='figures/' + fn, width='100%', height=800)

Questions to answer are:
1. How good are the fits after enforcing conditions on the parameters to make the whole surface arb free?
1. How is the fit after enforcing the model to be in between bid/ask prices? 
1. How come there is such a significant different between put & call slices? Put/Call parity, spread?
1. How do the fitting params behave intraday and across earnings releases. Can they be used for forecasting? 

In order to support fitting conditions and boundaries, refactoring to using pyomo ...